# ACLED Feature Services

In [180]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, FeatureCollection, FeatureSet, Feature
from arcgis.geometry import Point, MultiPoint, Polygon
from arcgis.geometry.filters import intersects
from arcgis.mapping.ogc import GeoJSONLayer
from arcgis.mapping.renderer import generate_renderer
from datetime import datetime, timedelta
from IPython.display import display
from ipywidgets import widgets
import os

In [2]:
acled_fs_url = os.environ['acled_feature_service_url']
acled_fs_url = acled_fs_url.replace('/1', '/0')
acled_feature_layer = FeatureLayer(acled_fs_url)

In [3]:
def query_by_date(layer, event_date, out_sr=None):
    start_date_str = from_date(event_date)
    end_date_str = from_date(event_date + timedelta(days=1))
    where_clause = "EVENT_DATE BETWEEN DATE '{0}' AND DATE '{1}'".format(start_date_str, end_date_str)
    print(where_clause)
    return layer.query(where_clause, out_sr=out_sr)

def query_by_date_range(layer, start_date, end_date, out_sr=None):
    start_date = from_date(start_date)
    end_date = from_date(end_date)
    where_clause = "EVENT_DATE BETWEEN DATE '{0}' AND DATE '{1}'".format(start_date, end_date)
    print(where_clause)
    return layer.query(where_clause, out_sr=out_sr)

def from_date(date):
    return date.strftime('%Y-%m-%d')
    
def to_date(date_as_string):
    return datetime.strptime(date_as_string, '%Y-%m-%d')

def find_events():
    #where_clause = "EVENT_DATE > DATE '2021-07-01' AND EVENT_TYPE='Battles'"
    #where_clause = "EVENT_DATE BETWEEN DATE '2021-07-01' AND DATE '2021-09-30' AND SUB_EVENT_TYPE = 'BATTLE'"
    #where_clause = "EVENT_DATE > DATE '2021-07-01' AND EVENT_DATE < DATE '2021-07-30'"
    event_date = to_date('2021-09-10')
    for query_indx in range(0, 10):
        acled_feature_set = query_by_date(acled_feature_layer, event_date)
        print(event_date, len(acled_feature_set))
        if 0 < len(acled_feature_set):
            break
    
        event_date += timedelta(days=-1)

In [4]:
start_date_picker = widgets.DatePicker(
    description='Pick start',
    disabled=False
)
display(start_date_picker)

end_date_picker = widgets.DatePicker(
    description='Pick end',
    disabled=False
)
display(end_date_picker)

DatePicker(value=None, description='Pick start')

DatePicker(value=None, description='Pick end')

In [228]:
#target_wkid = 4326
target_wkid = 3857
start_date = start_date_picker.value
end_date = end_date_picker.value
acled_feature_set = query_by_date_range(acled_feature_layer, start_date, end_date, out_sr=target_wkid)
acled_feature_set

EVENT_DATE BETWEEN DATE '2021-01-01' AND DATE '2021-09-30'


<FeatureSet> 3971 features

In [33]:
gis = GIS()

def get_europe_map():
    europe_map = gis.map("Europe")
    europe_map.basemap = "dark-gray-vector"
    return europe_map

def get_countries_layer():
    return FeatureLayer("https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/World_Countries_(Generalized)/FeatureServer/0")

In [34]:
europe_map = get_europe_map()
#europe_map.add_layer(acled_feature_layer)
europe_map.add_layer(acled_feature_set,  {"renderer": "HeatmapRenderer"})
europe_map

MapView(layout=Layout(height='400px', width='100%'))

In [35]:
acled_feature_set.sdf

,OBJECTID,data_id,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,actor1,assoc_actor_1,...,source,notes,fatalities,timestamp,iso,iso3,region,source_scale,sub_event_type,SHAPE
0,4185659,8583703.0,IND100834,100834,2021-09-30,2021,1,Riots,Rioters (India),IPFT: Indigenous Peoples Front of Tripura,...,United News of India,"On 30 September 2021, members of Indigenous Pe...",0,1.633468e+09,356,IND,South Asia,National,Mob violence,"{""x"": 10197499.87776138, ""y"": 2761574.47892884..."
1,4185660,8583664.0,IND100835,100835,2021-09-30,2021,1,Protests,Protesters (India),Farmers (India); BKU: Bharatiya Kisan Union,...,Times of India,"On 30 September 2021, hundreds of farmers asso...",0,1.633468e+09,356,IND,South Asia,National,Peaceful protest,"{""x"": 8638503.805048823, ""y"": 3280858.35954545..."
2,4185661,8583653.0,IND100837,100837,2021-09-30,2021,1,Riots,Rioters (India),TMC: Trinamool Congress Party,...,Times of India,"On 30 September 2021, a clash broke out betwee...",0,1.633468e+09,356,IND,South Asia,National,Mob violence,"{""x"": 9837826.603008315, ""y"": 2580437.12881159..."
3,4185662,8583649.0,IND100826,100826,2021-09-30,2021,1,Protests,Protesters (India),MNS: Maharashtra Navnirman Sena,...,Times of India,"On 30 September 2021, MNS members staged a pro...",0,1.633468e+09,356,IND,South Asia,National,Peaceful protest,"{""x"": 8803690.797436962, ""y"": 2409331.87368333..."
4,4185663,8583644.0,IND100832,100832,2021-09-30,2021,1,Protests,Protesters (India),ABVP: Akhil Bharatiya Vidyarthi Parishad; Stud...,...,Times of India,"On 30 September 2021, students under the ABVP ...",0,1.633468e+09,356,IND,South Asia,National,Protest with intervention,"{""x"": 8436592.512647983, ""y"": 3112756.25642090..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3966,4189625,NaN,USA35159,35159,2021-09-30,2021,1,Protests,Protesters (United States),,...,7 News,"On 30 September 2021, a group of people gather...",0,NaN,840,USA,North America,Subnational,Peaceful protest,"{""x"": -9172058.124420984, ""y"": 4143922.3317586..."
3967,4189626,NaN,USA35146,35146,2021-09-30,2021,1,Protests,Protesters (United States),Labour Group (United States),...,ABC7 (San Francisco),"On 30 September 2021, hundreds of Kaiser engin...",0,NaN,840,USA,North America,Subnational,Peaceful protest,"{""x"": -13611145.458784355, ""y"": 4551774.467367..."
3968,4189627,NaN,USA35154,35154,2021-09-30,2021,1,Protests,Protesters (United States),Students (United States),...,CBS19 (Shaker Heights),"On 30 September 2021, students came together i...",0,NaN,840,USA,North America,Subnational,Peaceful protest,"{""x"": -9074653.569976868, ""y"": 5024296.4966427..."
3969,4189628,NaN,USA35148,35148,2021-09-30,2021,1,Protests,Protesters (United States),Fight for 15; Labour Group (United States),...,Fox35,"On 30 September 2021, workers affiliated with ...",0,NaN,840,USA,North America,Subnational,Peaceful protest,"{""x"": -9059959.397192156, ""y"": 3316594.0346150..."


In [229]:
import geopandas
import json

In [230]:
hexbins_file_path = os.environ['hexbins_file_path']
hexbins = geopandas.read_file(hexbins_file_path)

In [231]:
hexbins

,id,left,top,right,bottom,geometry
0,1,-2.003751e+07,2.003751e+07,-1.974883e+07,1.978751e+07,"POLYGON ((-20037508.343 19912508.343, -1996533..."
1,2,-2.003751e+07,1.978751e+07,-1.974883e+07,1.953751e+07,"POLYGON ((-20037508.343 19662508.343, -1996533..."
2,3,-2.003751e+07,1.953751e+07,-1.974883e+07,1.928751e+07,"POLYGON ((-20037508.343 19412508.343, -1996533..."
3,4,-2.003751e+07,1.928751e+07,-1.974883e+07,1.903751e+07,"POLYGON ((-20037508.343 19162508.343, -1996533..."
4,5,-2.003751e+07,1.903751e+07,-1.974883e+07,1.878751e+07,"POLYGON ((-20037508.343 18912508.343, -1996533..."
...,...,...,...,...,...,...
29941,29942,2.001617e+07,-1.908749e+07,2.030484e+07,-1.933749e+07,"POLYGON ((20016166.582 -19212491.657, 20088335..."
29942,29943,2.001617e+07,-1.933749e+07,2.030484e+07,-1.958749e+07,"POLYGON ((20016166.582 -19462491.657, 20088335..."
29943,29944,2.001617e+07,-1.958749e+07,2.030484e+07,-1.983749e+07,"POLYGON ((20016166.582 -19712491.657, 20088335..."
29944,29945,2.001617e+07,-1.983749e+07,2.030484e+07,-2.008749e+07,"POLYGON ((20016166.582 -19962491.657, 20088335..."


In [232]:
acled_locations = geopandas.GeoDataFrame.from_features(json.loads(acled_feature_set.to_geojson)["features"], crs=target_wkid)

In [233]:
acled_locations

,geometry,OBJECTID,data_id,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,actor1,...,geo_precision,source,notes,fatalities,timestamp,iso,iso3,region,source_scale,sub_event_type
0,POINT (10197499.878 2761574.479),4185659,8583703.0,IND100834,100834,1632960000000,2021,1,Riots,Rioters (India),...,2,United News of India,"On 30 September 2021, members of Indigenous Pe...",0,1.633468e+09,356,IND,South Asia,National,Mob violence
1,POINT (8638503.805 3280858.360),4185660,8583664.0,IND100835,100835,1632960000000,2021,1,Protests,Protesters (India),...,1,Times of India,"On 30 September 2021, hundreds of farmers asso...",0,1.633468e+09,356,IND,South Asia,National,Peaceful protest
2,POINT (9837826.603 2580437.129),4185661,8583653.0,IND100837,100837,1632960000000,2021,1,Riots,Rioters (India),...,1,Times of India,"On 30 September 2021, a clash broke out betwee...",0,1.633468e+09,356,IND,South Asia,National,Mob violence
3,POINT (8803690.797 2409331.874),4185662,8583649.0,IND100826,100826,1632960000000,2021,1,Protests,Protesters (India),...,1,Times of India,"On 30 September 2021, MNS members staged a pro...",0,1.633468e+09,356,IND,South Asia,National,Peaceful protest
4,POINT (8436592.513 3112756.256),4185663,8583644.0,IND100832,100832,1632960000000,2021,1,Protests,Protesters (India),...,1,Times of India,"On 30 September 2021, students under the ABVP ...",0,1.633468e+09,356,IND,South Asia,National,Protest with intervention
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3966,POINT (-9172058.124 4143922.332),4189625,NaN,USA35159,35159,1632960000000,2021,1,Protests,Protesters (United States),...,1,7 News,"On 30 September 2021, a group of people gather...",0,NaN,840,USA,North America,Subnational,Peaceful protest
3967,POINT (-13611145.459 4551774.467),4189626,NaN,USA35146,35146,1632960000000,2021,1,Protests,Protesters (United States),...,1,ABC7 (San Francisco),"On 30 September 2021, hundreds of Kaiser engin...",0,NaN,840,USA,North America,Subnational,Peaceful protest
3968,POINT (-9074653.570 5024296.497),4189627,NaN,USA35154,35154,1632960000000,2021,1,Protests,Protesters (United States),...,1,CBS19 (Shaker Heights),"On 30 September 2021, students came together i...",0,NaN,840,USA,North America,Subnational,Peaceful protest
3969,POINT (-9059959.397 3316594.035),4189628,NaN,USA35148,35148,1632960000000,2021,1,Protests,Protesters (United States),...,1,Fox35,"On 30 September 2021, workers affiliated with ...",0,NaN,840,USA,North America,Subnational,Peaceful protest


In [234]:
hexbins_joined = geopandas.sjoin(acled_locations, hexbins, how="left")

/home/gisfromscratch/anaconda3/lib/python3.8/site-packages/geopandas/tools/sjoin.py:58: UserWarning: CRS of frames being joined does not match!(3857 != {'init': 'epsg:3857'})
  warn(


In [235]:
hexbins_joined["hit_count"] = 1
hexbins_dissolved = hexbins_joined.dissolve(by="index_right", aggfunc="count")
hexbins.loc[hexbins_dissolved.index, "hit_count"] = hexbins_dissolved["hit_count"].values
hexbins["hit_count"] = hexbins["hit_count"].fillna(0.0).astype(int)

In [236]:
# GeoJSON expects WGS84 only!!!
def featureset_from_geodataframe(geo_df, spatial_ref):
    hexbins_featureset = FeatureSet.from_geojson(json.loads(geo_df.to_json()))
    hexbins_features = []
    for feature in hexbins_featureset.features:
        # From counter-clockwise to clockwise oriented rings
        rings = Polygon(feature.geometry).coordinates()[:,::-1,:].tolist()
        geometry = {"rings": rings, "spatialReference": spatial_reference}
        attributes = {}
        for attribute_name in feature.attributes:
            attributes[attribute_name] = feature.attributes[attribute_name]
        hexbins_feature = Feature(geometry=geometry, attributes=attributes)
        hexbins_features.append(hexbins_feature)
    return FeatureSet(hexbins_features, spatial_reference=spatial_reference)

spatial_reference = {"wkid": 102100, "latestWkid": 3857}
geo_df = hexbins[hexbins["hit_count"] > 0]
hexbins_featureset = featureset_from_geodataframe(geo_df, spatial_reference)
hexbins_featureset.display_field_name = "hit_count"

#hexbins_layer = GeoJSONLayer(data = json.loads(hexbins[hexbins["hit_count"] > 10].to_json()))

In [237]:
europe_map = get_europe_map()
#'''
hexbins_featureset.sdf.spatial.plot(europe_map,
    renderer_type="c",  # for class breaks renderer
    method="esriClassifyNaturalBreaks",  # classification algorithm
    class_count=5,  # choose the number of classes
    col="hit_count",  # numeric column to classify
    cmap="coolwarm",  # color map to pick colors from for each class
    alpha=0.35  # specify opacity
)
#'''
#hexbins_renderer = generate_renderer("Polygon", hexbins_featureset.sdf, renderer_type="s", symbol_style="+")
#europe_map.add_layer(hexbins_featureset, { "type":"FeatureLayer", "renderer":"ClassedColorRenderer", "field_name":"hit_count" })
europe_map

MapView(layout=Layout(height='400px', width='100%'))